In [1]:
import xarray as xr
import numpy as np
from climatico import enso
import matplotlib.pyplot as plt
import cftime
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from climatico.util import weighted_mean, pacific_lon
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from shapely.geometry.polygon import Polygon

In [2]:
from config import directory_figs, directory_data

In [3]:
def grab_enso_spatial(ds, indices, year1, year2, filename, cutoff, filevar='SST'):
    """
    Use the input seasonal SST file ('QS-DEC') and precomputed Nino-index to extract Nino(a) 
    event indices in time array of SST file.
    Then extract the variable associated with the Nino(a) events. These focus on DJF months.
    """
    ### nino
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    nino = sv_indices.mean(dim='time')
    print("Nino done")

    ### nina
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    nina = sv_indices.mean(dim='time')
    print("Nina done")
    
    return nino, nina

In [3]:
# list of filenames to do this for:
# SSTs
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'
# wind stress x-direction
taux_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.cam.h0.TAUX.*.nc'
taux_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.cam.h0.TAUX.*.nc'
taux_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.cam.h0.TAUX.*.nc'
taux_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.cam.h0.TAUX.*.nc'
taux_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.cam.h0.TAUX.*.nc'
taux_cntrl = 'b.e11.B1850C5CN.f09_g16.005.cam.h0.TAUX.*.nc'
# wind stress y-direction
tauy_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.cam.h0.TAUY.*.nc'
tauy_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.cam.h0.TAUY.*.nc'
tauy_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.cam.h0.TAUY.*.nc'
tauy_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.cam.h0.TAUY.*.nc'
tauy_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.cam.h0.TAUY.*.nc'
tauy_cntrl = 'b.e11.B1850C5CN.f09_g16.005.cam.h0.TAUY.*.nc'

In [4]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)

In [5]:
#%%capture
ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0)
print("indices started")
# compute indices
control_nino = nino.compute_index(ds2['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
control_index = control_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
control_nino, control_nina = grab_enso_spatial(
                                              ds=ds2.resample(time='QS-DEC').mean(), 
                                              indices=control_index, 
                                              year1=801, 
                                              year2=1599, 
                                              filename=file_cntrl,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_cntrl}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
control_ninotx, control_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=control_index, 
                                              year1=801, 
                                              year2=1599, 
                                              filename=taux_cntrl,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_cntrl}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
control_ninoty, control_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=control_index, 
                                              year1=801, 
                                              year2=1599, 
                                              filename=tauy_cntrl,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 101 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [6]:
#%%capture
ds1 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)
print("indices started")
# compute indices
g02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
g02sv_index = g02sv_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
g02sv_nino, g02sv_nina = grab_enso_spatial(
                                              ds=ds1.resample(time='QS-DEC').mean(), 
                                              indices=g02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=file_g02sv,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_g02sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
g02sv_ninotx, g02sv_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=g02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=taux_g02sv,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_g02sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
g02sv_ninoty, g02sv_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=g02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=tauy_g02sv,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 51 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [7]:
#%%capture
ds1 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)
print("indices started")
# compute indices
g04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
g04sv_index = g04sv_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
g04sv_nino, g04sv_nina = grab_enso_spatial(
                                              ds=ds1.resample(time='QS-DEC').mean(), 
                                              indices=g04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=file_g04sv,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_g04sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
g04sv_ninotx, g04sv_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=g04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=taux_g04sv,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_g04sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
g04sv_ninoty, g04sv_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=g04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=tauy_g04sv,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [8]:
#%%capture
ds1 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)
print("indices started")
# compute indices
p02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
p02sv_index = p02sv_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
p02sv_nino, p02sv_nina = grab_enso_spatial(
                                              ds=ds1.resample(time='QS-DEC').mean(), 
                                              indices=p02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=file_p02sv,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_p02sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
p02sv_ninotx, p02sv_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=p02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=taux_p02sv,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_p02sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
p02sv_ninoty, p02sv_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=p02sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=tauy_p02sv,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [9]:
#%%capture
ds1 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)
print("indices started")
# compute indices
p04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
p04sv_index = p04sv_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
p04sv_nino, p04sv_nina = grab_enso_spatial(
                                              ds=ds1.resample(time='QS-DEC').mean(), 
                                              indices=p04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=file_p04sv,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_p04sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
p04sv_ninotx, p04sv_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=p04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=taux_p04sv,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_p04sv}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
p04sv_ninoty, p04sv_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=p04sv_index, 
                                              year1=201, 
                                              year2=500, 
                                              filename=tauy_p04sv,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [10]:
#%%capture

ds1 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                        combine='by_coords',
                        preprocess=nino.nino)
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)
print("indices started")
# compute indices
psalt_nino = nino.compute_index(ds1['SST'].groupby('time.month'), ds2['SST'].groupby('time.month').mean(), linear_detrend=False, lat_name='lat')
# grab numpy array
psalt_index = psalt_nino.resample(time='QS-DEC').mean().values
print("indices done")
# ssts
psalt_nino, psalt_nina = grab_enso_spatial(
                                              ds=ds1.resample(time='QS-DEC').mean(), 
                                              indices=psalt_index, 
                                              year1=101, 
                                              year2=250, 
                                              filename=file_psalt,
                                              cutoff=nino.cutoff, filevar='SST')
print("i")
# taux
dstx = xr.open_mfdataset(f'{directory_data}{taux_psalt}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
psalt_ninotx, psalt_ninatx = grab_enso_spatial(
                                              ds=dstx.resample(time='QS-DEC').mean(), 
                                              indices=psalt_index, 
                                              year1=101, 
                                              year2=250, 
                                              filename=taux_psalt,
                                              cutoff=nino.cutoff, filevar='TAUX')
print("ii")
# tauy
dsty = xr.open_mfdataset(f'{directory_data}{tauy_psalt}',
                         combine='by_coords',
                         preprocess=nino.nino)
# grab enso mean spatial maps
psalt_ninoty, psalt_ninaty = grab_enso_spatial(
                                              ds=dsty.resample(time='QS-DEC').mean(), 
                                              indices=psalt_index, 
                                              year1=101, 
                                              year2=250, 
                                              filename=tauy_psalt,
                                              cutoff=nino.cutoff, filevar='TAUY')
print("iii")

indices started


/glade/u/apps/ch/opt/python/3.7.9/gnu/9.1.0/pkg-library/20201220/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino done
Nina done
i
Nino done
Nina done
ii
Nino done
Nina done
iii


In [11]:
%%capture
cntrl_diff_nino = control_nino.values
cntrl_diff_ninotx = np.negative(control_ninotx).values
cntrl_diff_ninoty = np.negative(control_ninoty).values

In [12]:
%%capture
g02sv_diff_nino = (g02sv_nino-control_nino).values
g02sv_diff_ninotx = (np.negative(g02sv_ninotx)-np.negative(control_ninotx)).values
g02sv_diff_ninoty = (np.negative(g02sv_ninoty)-np.negative(control_ninoty)).values

In [13]:
%%capture
g04sv_diff_nino = (g04sv_nino-control_nino).values
g04sv_diff_ninotx = (np.negative(g04sv_ninotx)-np.negative(control_ninotx)).values
g04sv_diff_ninoty = (np.negative(g04sv_ninoty)-np.negative(control_ninoty)).values

In [14]:
%%capture
p02sv_diff_nino = (p02sv_nino-control_nino).values
p02sv_diff_ninotx = (np.negative(p02sv_ninotx)-np.negative(control_ninotx)).values
p02sv_diff_ninoty = (np.negative(p02sv_ninoty)-np.negative(control_ninoty)).values

In [15]:
%%capture
p04sv_diff_nino = (p04sv_nino-control_nino).values
p04sv_diff_ninotx = (np.negative(p04sv_ninotx)-np.negative(control_ninotx)).values
p04sv_diff_ninoty = (np.negative(p04sv_ninoty)-np.negative(control_ninoty)).values

In [16]:
%%capture
psalt_diff_nino = (psalt_nino-control_nino).values
psalt_diff_ninotx = (np.negative(psalt_ninotx)-np.negative(control_ninotx)).values
psalt_diff_ninoty = (np.negative(psalt_ninoty)-np.negative(control_ninoty)).values

In [17]:
data_assemble=xr.Dataset({
    
                 'cntrl_diff_nino':(['ssty','sstx'], cntrl_diff_nino),
                 'cntrl_diff_ninotx':(['y','x'], cntrl_diff_ninotx),
                 'cntrl_diff_ninoty':(['y','x'], cntrl_diff_ninoty),
    
                 'g02sv_diff_nino':(['ssty','sstx'], g02sv_diff_nino),
                 'g02sv_diff_ninotx':(['y','x'], g02sv_diff_ninotx),
                 'g02sv_diff_ninoty':(['y','x'], g02sv_diff_ninoty),
    
                 'g04sv_diff_nino':(['ssty','sstx'], g04sv_diff_nino),
                 'g04sv_diff_ninotx':(['y','x'], g04sv_diff_ninotx),
                 'g04sv_diff_ninoty':(['y','x'], g04sv_diff_ninoty),
    
                 'p02sv_diff_nino':(['ssty','sstx'], p02sv_diff_nino),
                 'p02sv_diff_ninotx':(['y','x'], p02sv_diff_ninotx),
                 'p02sv_diff_ninoty':(['y','x'], p02sv_diff_ninoty),
    
                 'p04sv_diff_nino':(['ssty','sstx'], p04sv_diff_nino),
                 'p04sv_diff_ninotx':(['y','x'], p04sv_diff_ninotx),
                 'p04sv_diff_ninoty':(['y','x'], p04sv_diff_ninoty),
    
                 'psalt_diff_nino':(['ssty','sstx'], psalt_diff_nino),
                 'psalt_diff_ninotx':(['y','x'], psalt_diff_ninotx),
                 'psalt_diff_ninoty':(['y','x'], psalt_diff_ninoty),
    
                },
                 coords=
                {'lon':(['sstx'], control_nino.coords['lon'].values),
                 'lat':(['ssty'], control_nino.coords['lat'].values),
                 'lons':(['x'], control_ninotx.coords['lon'].values),
                 'lats':(['y'], control_ninotx.coords['lat'].values)
                })     

In [18]:
data_assemble.to_netcdf(f'{directory_data}fig10_ninodata.nc')

In [19]:
%%capture
cntrl_diff_nina = control_nina.values
cntrl_diff_ninatx = np.negative(control_ninatx).values
cntrl_diff_ninaty = np.negative(control_ninaty).values

In [20]:
%%capture
g02sv_diff_nina = (g02sv_nina-control_nina).values
g02sv_diff_ninatx = (np.negative(g02sv_ninatx)-np.negative(control_ninatx)).values
g02sv_diff_ninaty = (np.negative(g02sv_ninaty)-np.negative(control_ninaty)).values

In [21]:
%%capture
g04sv_diff_nina = (g04sv_nina-control_nina).values
g04sv_diff_ninatx = (np.negative(g04sv_ninatx)-np.negative(control_ninatx)).values
g04sv_diff_ninaty = (np.negative(g04sv_ninaty)-np.negative(control_ninaty)).values

In [22]:
%%capture
p02sv_diff_nina = (p02sv_nina-control_nina).values
p02sv_diff_ninatx = (np.negative(p02sv_ninatx)-np.negative(control_ninatx)).values
p02sv_diff_ninaty = (np.negative(p02sv_ninaty)-np.negative(control_ninaty)).values

In [23]:
%%capture
p04sv_diff_nina = (p04sv_nina-control_nina).values
p04sv_diff_ninatx = (np.negative(p04sv_ninatx)-np.negative(control_ninatx)).values
p04sv_diff_ninaty = (np.negative(p04sv_ninaty)-np.negative(control_ninaty)).values

In [24]:
%%capture
psalt_diff_nina = (psalt_nina-control_nina).values
psalt_diff_ninatx = (np.negative(psalt_ninatx)-np.negative(control_ninatx)).values
psalt_diff_ninaty = (np.negative(psalt_ninaty)-np.negative(control_ninaty)).values

In [25]:
data_assemble=xr.Dataset({
    
                 'cntrl_diff_nina':(['ssty','sstx'], cntrl_diff_nina),
                 'cntrl_diff_ninatx':(['y','x'], cntrl_diff_ninatx),
                 'cntrl_diff_ninaty':(['y','x'], cntrl_diff_ninaty),
    
                 'g02sv_diff_nina':(['ssty','sstx'], g02sv_diff_nina),
                 'g02sv_diff_ninatx':(['y','x'], g02sv_diff_ninatx),
                 'g02sv_diff_ninaty':(['y','x'], g02sv_diff_ninaty),
    
                 'g04sv_diff_nina':(['ssty','sstx'], g04sv_diff_nina),
                 'g04sv_diff_ninatx':(['y','x'], g04sv_diff_ninatx),
                 'g04sv_diff_ninaty':(['y','x'], g04sv_diff_ninaty),
    
                 'p02sv_diff_nina':(['ssty','sstx'], p02sv_diff_nina),
                 'p02sv_diff_ninatx':(['y','x'], p02sv_diff_ninatx),
                 'p02sv_diff_ninaty':(['y','x'], p02sv_diff_ninaty),
    
                 'p04sv_diff_nina':(['ssty','sstx'], p04sv_diff_nina),
                 'p04sv_diff_ninatx':(['y','x'], p04sv_diff_ninatx),
                 'p04sv_diff_ninaty':(['y','x'], p04sv_diff_ninaty),
    
                 'psalt_diff_nina':(['ssty','sstx'], psalt_diff_nina),
                 'psalt_diff_ninatx':(['y','x'], psalt_diff_ninatx),
                 'psalt_diff_ninaty':(['y','x'], psalt_diff_ninaty),
    
                },
                 coords=
                {'lon':(['sstx'], control_nina.coords['lon'].values),
                 'lat':(['ssty'], control_nina.coords['lat'].values),
                 'lons':(['x'], control_ninatx.coords['lon'].values),
                 'lats':(['y'], control_ninatx.coords['lat'].values)
                })     

In [26]:
data_assemble.to_netcdf(f'{directory_data}fig11_ninadata.nc')